# Reference 
* [데이터 사이언스 스쿨 - 피봇테이블과 그룹 분석](https://datascienceschool.net/view-notebook/76dcd63bba2c4959af15bec41b197e7c/)

In [1]:
import pandas as pd
import numpy as np

# Pivot Table 정리

피봇 테이블이란 데이터 열(column) 중에서 두 개를 키(key)로 사용하여 데이터를 선택하는 방법
    
피봇 테이블을 사용하기 위해서는 키가 될 수 있는 두 개의 열(column) 혹은 필드(field)를 선택하여 이 두열을 
        * 행 인덱스(row index)
        * 열 인덱스(column index)
로 변경하고 행 조건과 열 조건에 맞는 데이터를 찾아서 해당 칸에 넣는다. 만약 주어진 데이터가 없으면 NaN 값을 넣는다.

In [2]:
data = {
    'state': ['Ohio', 'Ohio', 'Ohio', 'Nevada', 'Nevada'],
    'year': [2000, 2001, 2002, 2001, 2002],
    'pop': [1.5, 2.5, 3.0, 2.5, 3.5]
}
df = pd.DataFrame(data, columns=["state", "year", "pop"])
df

,state,year,pop
0,Ohio,2000,1.5
1,Ohio,2001,2.5
2,Ohio,2002,3.0
3,Nevada,2001,2.5
4,Nevada,2002,3.5


In [3]:
# 첫번째 인수 : 행 인덱스에서 사용할 column name,
# 두번째 인수 : 열 인덱스에서 사용할 column name,
# 세번째 인수 : 마지막으로 데이터로 사용할 column name
df.pivot("state", "year", "pop")

year,2000,2001,2002
state,,,
Nevada,NaN,2.5,3.5
Ohio,1.5,2.5,3.0


In [4]:
df.set_index(["state", "year"]).unstack()

pop          
year   2000 2001 2002
state                
Nevada  NaN  2.5  3.5
Ohio    1.5  2.5  3.0

# 그룹 분석
* 그룹 분석은 피봇 테이블가 달리 키에 의해서 결정되는 데이터가 복수개가 있어도 괜찮다. 대신 연산을 통해 복수개의 그룹데이터에 대한 대표값을 정한다.

* 그룹 연산을 하는 방법
    * 분석하고자 하는 Series나 DataFrame에 groupby 메서드를 호출
    * 호출한 결과에 그룹 연산을 수행
    
## groupby 메서드
* groupby 메서드는 데이터를 그룹 별로 분류하는 역할을 한다. groupby 메서드의 인수로는 다음과 같은 값을 사용
        1. 열 또는 열의 리스트
        2. 행 인덱스
* 연산 결과로 Groupby 클래스 객체를 반환하는데 이 객체에는 그룹별로 연산을 할 수 있는 그룹 연산 메서드가 있다.

## 그룹 연산 메서드

* 자주 사용되는 그룹 연산 메서드들
    * size(), count() : 갯수
    * mean(), median(), min(), max() : 평균, 중앙값, 최소, 최대
    * sum(), prod(), std(), quantile() : 합계, 곱, 표준편차, 사분위수
    * first(), last() : 가장 첫번째 데이터와 가장 나중 데이터
    
    * agg(), aggregate()
        * 만약 원하는 그룹 연산이 없는 경우 함수를 만들고 이 함수를 agg()에 전달
        * 또는 여러가지 그룹 연산을 동시에 하고 싶은 경우 함수 이름 문자열의 리스트를 전달
    * transform()
        * 그룹 연산으로 대푯값을 만든 다음, 이 대푯 값을 새로운 열(column)로 원래 데이터 프레임에 추가
    
    * describe()
        * 하나의 그룹 대푯값이 아니라 여러개의 값을 데이터프레임으로 구한다.
        
    * apply()
        * describe() 처럼 하나의 대푯값이 아닌 데이터프레임을 출력하지만 원하는 그룹연산이 없는 경우 사용

In [5]:
np.random.seed(0)
df = pd.DataFrame({'key1' : ['a', 'a', 'b', 'b', 'a'],
                   'key2' : ['one', 'two', 'one', 'two', 'one'],
                   'data1' : np.random.randn(5),
                   'data2' : np.random.randn(5)})
df

,data1,data2,key1,key2
0,1.764052,-0.977278,a,one
1,0.400157,0.950088,a,two
2,0.978738,-0.151357,b,one
3,2.240893,-0.103219,b,two
4,1.867558,0.410599,a,one


* key 1 값에 따른 data1의 평균?

In [6]:
df.groupby(df.key1).mean()

,data1,data2
key1,,
a,1.343923,0.127803
b,1.609816,-0.127288


* data1에 대해서만 하고싶다면? 미리 series를 구하거나 groupby 반환값 또는 최종 그룹연산 결과에서 data1만 뽑아도 된다.

In [7]:
df.data1.groupby(df.key1).mean()

key1
a    1.343923
b    1.609816
Name: data1, dtype: float64

In [8]:
df.groupby(df.key1)["data1"].mean()

key1
a    1.343923
b    1.609816
Name: data1, dtype: float64

In [9]:
df.groupby(df.key1).mean()["data1"]

key1
a    1.343923
b    1.609816
Name: data1, dtype: float64

* 복합 key(key1, key2) 값에 따른 data1의 평균 구하기. 이 문제는 피봇 테이블과 유사함. 분석하고자 하는 키가 여러개이면 리스트를 사용

In [10]:
df.data1.groupby([df.key1, df.key2]).mean()

key1  key2
a     one     1.815805
      two     0.400157
b     one     0.978738
      two     2.240893
Name: data1, dtype: float64

In [11]:
df.data1.groupby([df.key1, df.key2]).mean().unstack("key2")

key2,one,two
key1,,
a,1.815805,0.400157
b,0.978738,2.240893


### pivot_table

* pivot 명령과 groupby 명령의 중간적 성격을 가지는 pivot_table 명령
    * pivot_table 명령은 groupby 명령처럼 그룹 분석을 하지만 최종적으로는 pivot 명령처럼 피봇 테이블을 만든다. groupby 명령 결과에 unstack을 자동 적용하여 2차원적인 형태로 변형
    
    * pivot_table(data, values=None, index=None, columns=None, aggfunc='mean', fill_value=None, margins=False, margins_name='All')
    * data: 분석할 데이터프레임 (메서드일 때는 필요하지 않음)
    * values: 분석할 데이터프레임에서 분석할 열
    * index: 행 인덱스로 들어갈 키 열 또는 키 열의 리스트
    * columns: 열 인덱스로 들어갈 키 열 또는 키 열의 리스트
    * aggfunc: 분석 메서드
    * fill_value: NaN 대체 값
    * margins: 오른쪽과 아래에 합계를 붙일지 여부
    * margins_name: 합계 열(행)의 이름
    
    
* 따라서 일반 피봇 테이블의 관점에서 볼 때는 pivot을 수행하지만 데이터가 유니크하게 선택되지 않으면 aggfunc 인수로 정의된 함수를 수행하여 대표값을 계산하는 것과 같다.

In [12]:
df.pivot_table("data1", "key1", "key2")

key2,one,two
key1,,
a,1.815805,0.400157
b,0.978738,2.240893


In [25]:
df.pivot_table(["data1","data2"], "key1","key2")

data1               data2          
key2       one       two       one       two
key1                                        
a     1.815805  0.400157 -0.283340  0.950088
b     0.978738  2.240893 -0.151357 -0.103219

##  Tip 데이터 예제

In [26]:
import seaborn as sns

In [27]:
tips = sns.load_dataset("tips")
tips.tail()

,total_bill,tip,sex,smoker,day,time,size
239,29.03,5.92,Male,No,Sat,Dinner,3
240,27.18,2.00,Female,Yes,Sat,Dinner,2
241,22.67,2.00,Male,Yes,Sat,Dinner,2
242,17.82,1.75,Male,No,Sat,Dinner,2
243,18.78,3.00,Female,No,Thur,Dinner,2



* 이 데이터프레임에서 각각의 컬럼은 다음을 뜻한다.

    * total_bill: 식사대금
    * tip: 팁
    * sex: 성별
    * smoker: 흡연/금연 여부
    * day: 요일
    * time: 시간
    * size: 인원

* 식사대금과 팁의 비율을 나타내는 tip_pct 추가

In [28]:
tips['tip_pct'] = tips['tip'] / tips['total_bill']
tips.tail()

,total_bill,tip,sex,smoker,day,time,size,tip_pct
239,29.03,5.92,Male,No,Sat,Dinner,3,0.203927
240,27.18,2.00,Female,Yes,Sat,Dinner,2,0.073584
241,22.67,2.00,Male,Yes,Sat,Dinner,2,0.088222
242,17.82,1.75,Male,No,Sat,Dinner,2,0.098204
243,18.78,3.00,Female,No,Thur,Dinner,2,0.159744


In [29]:
tips.describe()

,total_bill,tip,size,tip_pct
count,244.000000,244.000000,244.000000,244.000000
mean,19.785943,2.998279,2.569672,0.160803
std,8.902412,1.383638,0.951100,0.061072
min,3.070000,1.000000,1.000000,0.035638
25%,13.347500,2.000000,2.000000,0.129127
50%,17.795000,2.900000,2.000000,0.154770
75%,24.127500,3.562500,3.000000,0.191475
max,50.810000,10.000000,6.000000,0.710345


### 그룹별 통계
* 먼저 성별로 나누어 데이터 갯수 세어보기

In [30]:
tips.groupby("sex").count()

,total_bill,tip,smoker,day,time,size,tip_pct
sex,,,,,,,
Male,157,157,157,157,157,157,157
Female,87,87,87,87,87,87,87


In [31]:
tips.groupby("sex").size()

sex
Male      157
Female     87
dtype: int64

* 성별과 흡연유무로 나누어 데이터의 갯수와 팁 액수의 평균 알아보기

In [32]:
tips.groupby(["sex", "smoker"]).size()

sex     smoker
Male    Yes       60
        No        97
Female  Yes       33
        No        54
dtype: int64

* 피봇 테이블 형태로 만들기

In [33]:
tips.pivot_table("tip_pct", "sex", "smoker", aggfunc="count", margins=True)

smoker,Yes,No,All
sex,,,
Male,60.0,97.0,157.0
Female,33.0,54.0,87.0
All,93.0,151.0,244.0


In [34]:
tips.pivot_table("tip_pct", "sex", "smoker")

smoker,Yes,No
sex,,
Male,0.152771,0.160669
Female,0.182150,0.156921


* 성별과 흡연유무로 나누어 팁 액수의 평균뿐 아니라 여러가지 통곗값 알아보기

In [38]:
tips.groupby(["sex", "smoker"])[["tip", "tip_pct"]].describe().stack()

tip    tip_pct
sex    smoker                            
Male   Yes    count  60.000000  60.000000
              mean    3.051167   0.152771
              std     1.500120   0.090588
              min     1.000000   0.035638
              25%     2.000000   0.101845
              50%     3.000000   0.141015
              75%     3.820000   0.191697
              max    10.000000   0.710345
       No     count  97.000000  97.000000
              mean    3.113402   0.160669
              std     1.489559   0.041849
              min     1.250000   0.071804
              25%     2.000000   0.131810
              50%     2.740000   0.157604
              75%     3.710000   0.186220
              max     9.000000   0.291990
Female Yes    count  33.000000  33.000000
              mean    2.931515   0.182150
              std     1.219916   0.071595
              min     1.000000   0.056433
              25%     2.000000   0.152439
              50%     2.880000   0.173913
              75%     3.500000   0.198216
              max     6.500000   0.416667
       No     count  54.000000  54.000000
              mean    2.773519   0.156921
              std     1.128425   0.036421
              min     1.000000   0.056797
              25%     2.000000   0.139708
              50%     2.680000   0.149691
              75%     3.437500   0.181630
              max     5.200000   0.252672

* 이번에는 각 그룹에서 가장 많은 팁과 가장 적은 팁의 차이 알아보기. 함수가 없으므로 만들어야 한다.

In [39]:
def peak_to_peak(x):
    return x.max() - x.min()

tips.groupby(["sex", "smoker"])[["tip"]].agg(peak_to_peak)

tip
sex    smoker      
Male   Yes     9.00
       No      7.75
Female Yes     5.50
       No      4.20

* 만약 여러가지 그룹연산을 동시에 하고 싶다면 다음과 같이 리스트를 넣는다.

In [40]:
tips.groupby(["sex", "smoker"]).agg(["mean", peak_to_peak])[['total_bill']]

total_bill             
                    mean peak_to_peak
sex    smoker                        
Male   Yes     22.284500        43.56
       No      19.791237        40.82
Female Yes     17.977879        41.23
       No      18.105185        28.58

* 데이터 열마다 다른 연산을 하고 싶다면 column 라벨과 연산 이름(또는 함수)를 dict 형태로 넣는다

In [41]:
tips.groupby(["sex", "smoker"]).agg({'tip_pct' : 'mean', 'total_bill' : peak_to_peak})

tip_pct  total_bill
sex    smoker                      
Male   Yes     0.152771       43.56
       No      0.160669       40.82
Female Yes     0.182150       41.23
       No      0.156921       28.58

* pivot_table 명령을 사용하면 좀 더 간단하게 코드를 작성할 수 있음.

In [46]:
tips.pivot_table(index=['sex', 'smoker'], aggfunc='mean')

size       tip   tip_pct  total_bill
sex    smoker                                          
Male   Yes     2.500000  3.051167  0.152771   22.284500
       No      2.711340  3.113402  0.160669   19.791237
Female Yes     2.242424  2.931515  0.182150   17.977879
       No      2.592593  2.773519  0.156921   18.105185

In [47]:
tips.pivot_table(['tip_pct', 'size'], index=['sex', 'day'], columns='smoker')

size             tip_pct          
smoker            Yes        No       Yes        No
sex    day                                         
Male   Thur  2.300000  2.500000  0.164417  0.165706
       Fri   2.125000  2.000000  0.144730  0.138005
       Sat   2.629630  2.656250  0.139067  0.162132
       Sun   2.600000  2.883721  0.173964  0.158291
Female Thur  2.428571  2.480000  0.163073  0.155971
       Fri   2.000000  2.500000  0.209129  0.165296
       Sat   2.200000  2.307692  0.163817  0.147993
       Sun   2.500000  3.071429  0.237075  0.165710

In [49]:
tips.pivot_table('size', index=['time', 'sex', 'smoker'],
                columns='day', aggfunc='sum')

day                   Thur   Fri   Sat    Sun
time   sex    smoker                         
Lunch  Male   Yes     23.0   5.0   NaN    NaN
              No      50.0   NaN   NaN    NaN
       Female Yes     17.0   6.0   NaN    NaN
              No      60.0   3.0   NaN    NaN
Dinner Male   Yes      NaN  12.0  71.0   39.0
              No       NaN   4.0  85.0  124.0
       Female Yes      NaN   8.0  33.0   10.0
              No       2.0   2.0  30.0   43.0

In [48]:
tips.pivot_table('size', index=['time', 'sex', 'smoker'],
                columns='day', aggfunc='sum', fill_value=0)

day                   Thur  Fri  Sat  Sun
time   sex    smoker                     
Lunch  Male   Yes       23    5    0    0
              No        50    0    0    0
       Female Yes       17    6    0    0
              No        60    3    0    0
Dinner Male   Yes        0   12   71   39
              No         0    4   85  124
       Female Yes        0    8   33   10
              No         2    2   30   43